This notebook prepares the data for the subsequent notebook `1-Step-Entropy-Analyze.ipynb`, which generates figures illustrating the uncertainty analysis of the wave equation, as described in Supplementary Material Section 3.


In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
num_devices = torch.cuda.device_count()
print("Number of visible GPUs:", num_devices)

for i in range(num_devices):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

current_device = torch.cuda.current_device()
print("Current device index:", current_device)
print("Current device name:", torch.cuda.get_device_name(current_device))

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from wave_equation import (
    compute_exact_solution_random_ic_vary_Nx,
    visualize_spline_ic,
    plot_both_grids,
)

from data_processing import (
    SimpleSerializerSettings,
    scale_2d_array,
    serialize_2d_integers,
    extract_training_and_test
)

from llama_utils import load_model_and_tokenizer, generate_text_multiple
MODEL_NAME = "meta-llama/Llama-3.1-8B"
# MODEL_NAME = "meta-llama/Llama-3.2-3B"
# MODEL_NAME = "meta-llama/Llama-3.2-1B"
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [ ]:
model, tokenizer = load_model_and_tokenizer(MODEL_NAME)

In [ ]:
# Define parameters for the Heat equation solver
L = 2       # Length of the spatial domain
c = 0.2     # Wave speed
T = 0.5     # Total simulation times
Nx = 14     # Number of spatial steps (excluding boundary points)
Nt = 20     # Number of time steps 
dx = L/(Nx+1)
dt = T/Nt

# Example: Demonstrating the process of generating and visualizing a random initial condition
init_cond_random = np.random.uniform(-0.5, 0.5, size=Nx)
fig = visualize_spline_ic(L, Nx, init_cond_random)
plt.tight_layout()
plt.show()

# Example: Demonstrating how to resample spatial points from an underlying random initial condition
Nx_original = Nx
Nx_new = 14
fig, cs, init_cond_random_new = plot_both_grids(L, Nx_original, Nx_new, init_cond_random)
plt.tight_layout()
plt.show()

# Serialization setup
settings = SimpleSerializerSettings(space_sep=",", time_sep=";")

In [ ]:
def calculate_entropies(generation_outputs, Nx):
    """
    Calculate entropy values from generation outputs.
    Output:
        entropies: Array of shape (Nx, n_future_steps) containing entropy values
        avg_entropy: Average entropy across spatial points
    """
    n_future_steps = len(generation_outputs)
    entropies = np.zeros((Nx, n_future_steps))
    
    # Calculate entropy for ALL spatial points
    for time_idx in range(n_future_steps):
        gen_output = generation_outputs[time_idx]
        for grid_idx in range(Nx):
            token_position = grid_idx * 2  # accounting for spatial separators
            if token_position < len(gen_output.scores):
                logits = gen_output.scores[token_position][0]
                p = torch.softmax(logits, -1).clamp_min(1e-30)
                entropy = -(p * torch.log(p)).sum().item()
                entropies[grid_idx, time_idx] = entropy
    # Calculate average entropy across all spatial points
    avg_entropy = entropies.mean(axis=0)
    return entropies, avg_entropy

In [ ]:
all_Nt_values = range(2, 41, 2)
n_seeds = 50
all_mean_entropies = []
all_std_entropies = []
all_se_entropies = []

for Nt in tqdm(all_Nt_values):
    dt = T / Nt
    avg_entropies_for_this_Nt = []
    for seed in range(n_seeds):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
        init_cond_random_seed = np.random.uniform(-0.5, 0.5, size=Nx)
        # Create new cubic spline for this initial condition
        fig, cs_seed, _ = plot_both_grids(L, Nx, Nx, init_cond_random_seed)
        plt.close(fig)
        # Compute exact solution with this initial condition
        u_exact = compute_exact_solution_random_ic_vary_Nx(L, c, T, Nx, Nt, spline_obj=cs_seed, spline_vel_obj=None)
        u_exact_scaled, vmin_exact, vmax_exact = scale_2d_array(u_exact)
        u_exact_serialized = serialize_2d_integers(u_exact_scaled, settings)
        input_time_steps = Nt
        train_serial, test_serial = extract_training_and_test(
            u_exact_serialized, input_time_steps, settings
        )
        # LLM prediction
        generated_token, gen_output = generate_text_multiple(
            prompt=train_serial,
            model=model,
            tokenizer=tokenizer,
            Nx=Nx,
        )
        # Calculate entropy
        entropies, avg_entropy = calculate_entropies([gen_output], Nx)
        avg_entropies_for_this_Nt.append(avg_entropy[0])
    avg_entropies_array = np.array(avg_entropies_for_this_Nt)
    mean_entropy = np.mean(avg_entropies_array)
    std_entropy = np.std(avg_entropies_array, ddof=1)
    se_entropy = std_entropy / np.sqrt(n_seeds)
    all_mean_entropies.append(mean_entropy)
    all_std_entropies.append(std_entropy)
    all_se_entropies.append(se_entropy)
mean_entropies = np.array(all_mean_entropies)
std_entropies = np.array(all_std_entropies)
se_entropies = np.array(all_se_entropies)

np.savez_compressed(
    "8B_1_step_token_time_variation.npz",
    mean_entropies_8B=mean_entropies,
    std_entropies_8B=std_entropies,
    se_entropies_8B=se_entropies,
)

In [ ]:
all_Nx_values = range(2, 41, 2)
fixed_Nt = 50
all_mean_entropies = []
all_std_entropies = []
all_se_entropies = []
# Reference resolution for generating initial conditions
Nx_reference = 14

for Nx in tqdm(all_Nx_values):
    dt = T / fixed_Nt
    avg_entropies_for_this_Nx = []
    
    for seed in range(n_seeds):
        # Set random seed for generating different initial conditions
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
        # Generate a new random initial condition at reference resolution
        init_cond_random_seed = np.random.uniform(-0.5, 0.5, size=Nx_reference)
        # Resample the initial condition from reference resolution to current Nx
        fig, cs_seed, init_cond_resampled = plot_both_grids(L, Nx_reference, Nx, init_cond_random_seed)
        plt.close(fig)
        # Compute exact solution for this Nx with the resampled initial condition
        u_exact = compute_exact_solution_random_ic_vary_Nx(L, c, T, Nx, fixed_Nt, spline_obj=cs_seed, spline_vel_obj=None)
        u_exact_scaled, vmin_exact, vmax_exact = scale_2d_array(u_exact)
        u_exact_serialized = serialize_2d_integers(u_exact_scaled, settings)
        input_time_steps = fixed_Nt
        train_serial, test_serial = extract_training_and_test(
            u_exact_serialized, input_time_steps, settings
        )
        # LLM prediction
        generated_token, gen_output = generate_text_multiple(
            prompt=train_serial,
            model=model,
            tokenizer=tokenizer,
            Nx=Nx,
        )
        # Calculate entropy
        entropies, avg_entropy = calculate_entropies([gen_output], Nx)
        avg_entropies_for_this_Nx.append(avg_entropy[0])
    avg_entropies_array = np.array(avg_entropies_for_this_Nx)
    mean_entropy = np.mean(avg_entropies_array)
    std_entropy = np.std(avg_entropies_array, ddof=1)
    se_entropy = std_entropy / np.sqrt(n_seeds)
    all_mean_entropies.append(mean_entropy)
    all_std_entropies.append(std_entropy)
    all_se_entropies.append(se_entropy)
mean_entropies_Nx = np.array(all_mean_entropies)
std_entropies_Nx = np.array(all_std_entropies)
se_entropies_Nx = np.array(all_se_entropies)

np.savez_compressed(
    "8B_1_step_token_space_variation.npz",
    mean_entropies_8B_Nx=mean_entropies_Nx,
    std_entropies_8B_Nx=std_entropies_Nx,
    se_entropies_8B_Nx=se_entropies_Nx,
)